# Ensembles: From Decision Trees to Extra Trees

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor

cars = pd.read_csv('/Users/gdamico/Desktop/cars.csv')

In [1]:
cars.shape

In [2]:
cars.tail()

In [3]:
# Let's check the datatypes of these columns.

cars.dtypes

In [4]:
# Let's look for nulls.

cars.isnull().sum()

In [ ]:
# Great! Let's map the cubicinches values to floats.

In [5]:
# What happened?



In [28]:
# Fix the problem!



In [6]:
# What about the weight column?



In [7]:
# Fixing



In [8]:
# Let's check the correlations of the other features with MPG.



## Fitting a Decision Tree

In [12]:
X = cars.drop(['mpg', ' brand'], axis=1)
y = cars['mpg']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=1)

rt = DecisionTreeRegressor(random_state=1, max_depth=2)
rt.fit(X_train, y_train)

In [13]:
rt.score(X_test, y_test)

In [14]:
rt.feature_importances_

In [18]:
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.externals.six import StringIO
from IPython.display import Image

In [15]:
dot_file = StringIO()

export_graphviz(rt, out_file=dot_file, filled=True, rounded=True)

image = pydotplus.graph_from_dot_data(dot_file.getvalue())
Image(image.create_png())

A single decision tree will often overfit your training data. There are steps one can take to help with this, like limiting the "depth" of the nodes. But it's often better to do something else: Plant another tree!

Of course, if a second tree is going to be of any value, it has to be *different from* the first. Here's a good algorithm for achieving that:

## Fitting a Set of Bagged Decision Trees

### Bagging Algorithm

Take a sample of your X_train and fit a decision tree to it. <br/>
Replace the first batch of data and repeat. <br/>
When you've got as many trees as you like, make use of all your individual trees' predictions to come up with some holistic prediction. (Most obviously, we could take the average of our predictions, but there are other methods we might try.)

<br/>

Because we're resampling our data with replacement, we're *bootstrapping*. <br/>
Because we're making use of our many samples' predictions, we're *aggregating*. <br/>
Because we're bootstrapping and aggregating all in the same algorithm, we're *bagging*.

In [16]:
# Instantiate a bagging regressor



In [17]:
# Fit it.



In [18]:
# Score it.



That's a significant improvement in $R^2$! Let's see if we can do even better.

## Fitting a Random Forest

### Random Forest Algorithm

Let's add an extra layer of randomization: Instead of using *all* the features of my model to optimize a branch at each node, I'll just choose a subset of my features.

In [19]:
# Let's try a forest with 100 trees.



In [20]:
# Score it.



## Fitting a Stand of Extremely Randomized Trees

### Extra Trees Algorithm

Sometimes we might want even one more bit of randomization. Instead of always choosing the *optimal* branching path, we might just choose a branching path at random. If we're doing that, then we've got extremely randomized trees.

In [21]:
# Again let's try 100 trees.



In [22]:
# Scoring



In [24]:
# Checking the feature importances



## Gridsearching

In [25]:
# Define param grid.



In [26]:
# Initialize the gridsearch object with five-fold cross-validation.



In [27]:
# Fit it.



In [28]:
# Score it.



In [29]:
# Get the best parameter values!



In [30]:
# And the best score



In [31]:
# And the best estimator



## Building a Model that Takes Raw Data as Input

Suppose we go with the best estimator according to our gridsearch results. If we want our model to be able to make predictions of MPG for an uncleaned row of input, we'll need to be able to clean the row before modeling.

The main cleaning moves that we made above were to fix the problems in the cubicinches and the weightlbs columns. We'll also need to drop the columns that don't belong in the model.

Let's write functions that will take care of those problems.

In [ ]:
# Reloading the dataset

cars_relo = pd.read_csv('/Users/gdamico/Desktop/cars.csv')

# Re-splitting

X_train2, X_test2, y_train2, y_test2 = \
train_test_split(cars_relo.drop('mpg', axis=1), cars_relo['mpg'])

In [ ]:
def clean(df):
    for col in [' cubicinches', ' weightlbs']:
        df[col].replace(' ', np.nan, inplace=True)
        df[col] = df[col].map(float)
        df[col].replace(np.nan, df[col].mean(), inplace=True)
    return df

In [ ]:
def drop(df):
    return df.drop(' brand', axis=1)

In [ ]:
def model_predict(df):
    df_clean = clean(df)
    df_preds = drop(df)
    et = ExtraTreesRegressor(min_samples_leaf=3,
                       min_samples_split=2,
                       n_estimators=100).fit(X_train, y_train)
    return et.predict(df_preds).reshape(-1, 1)

In [ ]:
model_predict(X_test2)

In [ ]:
list(zip(model_predict(X_test2), y_test2))